In [12]:
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch import nn
import torch.nn.functional as F
import sys
sys.path.append("..")

# Load the data
df = pd.read_csv('few_merged_data_f849_tlast29_reordered_byR2.csv')
X = df.iloc[:, :849]  # First 849 columns are features
y = df.iloc[:, 849:]  # Last 29 columns are targets
from pl.model_impl import *

In [ ]:

# Standardize features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)


In [ ]:

# Load models and make predictions
predictions = []
for target_index in range(df.shape[1] - 849):
    # Initialize model
    model = SingleTargetNet(X.shape[1])
    
    # Load saved weights
    model.load_state_dict(torch.load(f'checkpoints/best_model_target_{target_index}.pth'))
    
    # Set to evaluation mode
    model.eval()
    
    # Make predictions
    with torch.no_grad():
        pred = model(X_tensor).numpy()
        predictions.append(pred)

# Stack predictions into array
predictions = np.hstack(predictions)

# Create a DataFrame for predictions
predictions_df = pd.DataFrame(predictions, columns=[f'Prediction_{i}' for i in range(predictions.shape[1])])

# Save predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

print(f"Predictions shape: {predictions.shape}")